In [1]:
import collections
import enum
import time
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import random
import produce_round_key
import AES as ENCRYPT

In [2]:
AES_Sbox = np.array([
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16
])

In [3]:
def generate_trace(trace_save, nor_trace_maxmin_save, trace_path, select_index, num_avg, number_from_select = 1):
    #=========================================================================================
    #enerate_trace（）：将原始的数据转换为训练可用的trace，控制trace的平均次数来控制SNR。
    #-----------------------------------------------------------------------------------------
    #输入：
    #---------------------------
    #trace_save：将要储存的未normalized的trace的路径和文件名。
    #nor_trace_maxmin_save：将要储存的normalized之后的trace的路径和文件名。
    #trace_path：读取的原始数据的路径。
    #select_index：被选中进行处理的未破损的原始数据的索引。
    #num_avg：一个trace要被平均的次数
    #number_from_select: 出自同一个原始数据的trace的个数。default=1.
    #----------------------------
    #输出：
    #----------------------------
    #
    #=========================================================================================
    
    Trace = []
    nor_trace_maxmin = []
    
    for data_index in tqdm(select_index[:], ncols=60):

        path = trace_path + str(data_index) + '.npy'
        all_trace = np.load(path)
#         print(all_trace.shape)
        #all_trace = all_trace[:,600:1400]
        if len(all_trace) >= num_avg:
            for num_select in range(number_from_select):
        
                #trace_select = random.sample(range(0, len(all_trace)), num_avg) #随机取trace
                trace_select = range(0, num_avg)                               #顺序取trace
                one_trace = all_trace[trace_select].mean(axis=0)    
                Trace.append(one_trace)
    
                # for max-min normalization
                MAX = np.max(one_trace)
                MIN = np.min(one_trace)            
                nor_one_trace_maxmin = (one_trace-MIN)/(MAX-MIN)
                nor_trace_maxmin.append(nor_one_trace_maxmin)
        
        elif len(all_trace) < num_avg:
            
            for num_select in range(number_from_select):
                nor_one_trace_maxmin = np.array(nor_trace_maxmin).mean(axis=0) 
                nor_trace_maxmin.append(nor_one_trace_maxmin)
                
                one_trace =np.array(Trace).mean(axis=0)    
                Trace.append(one_trace)
                print("WHY Warning: there is an empty trace file, please check it. We replaced this trace by using averaged trace. The number for this trace is: "+ str(path) )
    np.save(trace_save, Trace)  # for chipwhisperer and machine learning
    np.save(nor_trace_maxmin_save, nor_trace_maxmin)  # for machine learning
    print("=========== Trace Generated =============")
    

In [4]:
def generate_pt(pt_save, plaintext_path, select_index, num_avg, number_from_select = 1, num_subkey = 16):
    #=========================================================================================
    #enerate_trace（）：将原始的数据转换为训练可用的trace，控制trace的平均次数来控制SNR。
    #-----------------------------------------------------------------------------------------
    #输入：
    #---------------------------
    #pt_save：将要储存的pt的路径和文件名。
    #plaintext_path：读取的原始数据的路径。
    #select_index：被选中进行处理的未破损的原始数据的索引。
    #num_avg：一个trace要被平均的次数
    #number_from_select: 出自同一个原始数据的trace的个数。default=1.
    #number_subkey: subkey的个数。default=16.
    #----------------------------
    #输出：
    #----------------------------
    #
    #=========================================================================================
    
    plaintext = []
    
    with open(plaintext_path, "r") as f:
        for line in f.readlines():
            line = line.strip('\n')  # delete /n
            plaintext.append(line)
    
    #将16进制转换为10进制数据。
    for i in range(len(plaintext)):
    
        temp = []
    
        for j in range(num_subkey):
            
            byte = plaintext[i][2 * j: 2 * j + 2]
            dec = int(byte, 16)
            temp.append(dec)
    
        plaintext[i] = temp
#     print(plaintext)
    plaintext = np.array(plaintext)[select_index]
    
    #拓展plaintext,使得和出自同一个原始数据的trace一一对应。
    plaintext=plaintext.repeat(number_from_select,axis=0)
    
    np.save(pt_save, plaintext) 
    
    print("=========== Plaintexts Generated =============")
    
    
    return plaintext

In [5]:
def generate_key(roundkey_save, keylist_save, key_path, select_index, num_avg, number_from_select = 1, num_subkey = 16, num_round = 10):
    #=========================================================================================
    #enerate_key（）：将原始的数据转换为训练可用的key和roundkey
    #-----------------------------------------------------------------------------------------
    #输入：
    #---------------------------
    #roundkey10_save：将要储存的roundkey的路径和文件名。
    #keylist_save：将要储存的key的路径和文件名。
    #key_path：读取的原始数据的路径。
    #select_index：被选中进行处理的未破损的原始数据的索引。
    #num_avg：一个trace要被平均的次数
    #number_from_select: 出自同一个原始数据的trace的个数。default=1.
    #number_subkey: subkey的个数。default=16.
    #num_round: roundkey的round数。default=10.
    #----------------------------
    #输出：
    #----------------------------
    #key: 得到的key，用来后续制造ct
    #=========================================================================================
    
    key = []    
    key_list = []
    roundkey = []
    #----------------------to generate the original key--------------------------
    with open(key_path, 'r') as f:
        data = f.read()
    #print(data)
    #将16进制转换为10进制数据。
    for i in range(num_subkey):
        
        byte = data[2 * i: 2 * i + 2]
        #print(byte)
        dec = int(byte, 16)
        key.append(dec)
    #print(key)
    key = np.array(key)
    
    #将fixedkey拓展成和trace数相同，方便后续操作
    for i in tqdm(range(len(select_index)),ncols=60):
        key_list.append(key)
    key_list = np.array(key_list)
    key_list=key_list.repeat(number_from_select,axis=0)
    print(key_list)
    #np.save(keylist_save, key_list) 
    
    #----------------------to generate the round key--------------------------

    for i in tqdm(range(len(key_list)), ncols=60):
        one_roundkey = produce_round_key.keyScheduleRounds(key_list[i], 0, num_round)
        roundkey.append(one_roundkey)
    roundkey = np.array(roundkey)
    print(roundkey)
    #
    np.save(roundkey_save,roundkey)
    
    print("=========== Key Generated =============")
    
    return key

In [6]:
def generate_ct(ct_save, plaintext, key, num_subkey = 16):
    #=========================================================================================
    #enerate_ct()：将原始的数据转换为训练可用的ct。
    #-----------------------------------------------------------------------------------------
    #输入：
    #---------------------------
    #ct_save：将要储存的ct的路径和文件名。
    #plaintext_path：读取的原始数据的路径。
    #key: 原始key,用来和plaintext结合得到ct.
    #number_subkey: subkey的个数。default=16.
    #----------------------------
    #输出：
    #----------------------------
    #
    #=========================================================================================
    
    ciphertexts = []        
    str_key = ''
    
    for i in range(len(key)):
        str_key = str_key + format(key[i], '#04x').split('x')[-1]
        #str_key = str_key + format(key[0][i].item(), '#04x').split('x')[-1]
    
    
    for pt in tqdm(plaintext,ncols=60):
    
        str_pt = ''
    
        for i in range(len(pt)):
            str_pt = str_pt + format(pt[i], '#04x').split('x')[-1]
    
        KEY = str_key
        DATA = str_pt    
        aes = ENCRYPT.AES(mode='ecb', input_type='hex')
        ct = aes.encryption(DATA, KEY)   
        ciphertexts.append(ct)

    #将16进制转换为10进制数据。    
    for i in range(len(ciphertexts)):
    
        temp = []
    
        for j in range(num_subkey):
            byte = ciphertexts[i][2 * j: 2 * j + 2]
            dec = int(byte, 16)
            temp.append(dec)
    
        ciphertexts[i] = temp    
    ciphertexts = np.array(ciphertexts)
    
    print("=========== Ciphertexts Generated =============")
    
    
    np.save(ct_save,ciphertexts)

In [7]:
def trace_process(total_nunmber, select_number, initial_number, save_folder, load_folder, num_avg, number_from_select = 1, num_subkey = 16, num_round = 10):
    #=========================================================================================
    #trace_process（）：将原始的数据转换为训练可用的数据
    #-----------------------------------------------------------------------------------------
    #输入：
    #---------------------------
    #save_folder: 存储路径。
    #load_folder：加载路径。
    #total_nunmber:读取的数据总量
    #select_number：选取的数量，因为会有outlier，所以要有余量。
    #initial_number：开始选取的索引，default=0
    #num_avg：一个trace要被平均的次数
    #number_from_select: 出自同一个原始数据的trace的个数。default=1.
    #number_subkey: subkey的个数。default=16.
    #num_round: roundkey的round数。default=10.
    #----------------------------
    #输出：
    #----------------------------
    #
    #=========================================================================================   
    
    
    #-------------------------定义存储路径-----------------------
    
    pt_save = save_folder + str(num_avg) + '_'+'pt'+'.npy'
    ct_save = save_folder+ str(num_avg) +'_'+'ct'+'.npy'
    roundkey10_save = save_folder + str(num_avg)+'_'+str(num_round)+'th_roundkey'+'.npy'
    keylist_save = save_folder + str(num_avg)+ '_'+'key'+'.npy'
    trace_save = save_folder +str(num_avg)+ '_traces'+'.npy'
    nor_trace_maxmin_save = save_folder + str(num_avg)+'_traces_maxmin'+'.npy'

    
    
    #-----------------------定义加载路径-------------------------
    mis_index_path = load_folder + '/mis_index.npy'
    plaintext_path = load_folder + '/pt_.txt'
    key_path = load_folder + '/key_.txt'
    trace_path = load_folder + '/all__'
    

    total_index = [i for i in range(initial_number,total_nunmber)]
    
    
    missing_index = np.load(mis_index_path)


    real_index = [x for x in total_index if x not in missing_index]
    real_index = np.array(real_index)   #np.array(real_index)
    select_index = real_index[[i for i in range(initial_number,select_number)]]
    
    generate_trace(trace_save, nor_trace_maxmin_save, trace_path, select_index, num_avg, number_from_select)
    plaintext = generate_pt(pt_save, plaintext_path, select_index, num_avg, number_from_select, num_subkey = 16)
    key = generate_key(roundkey10_save, keylist_save, key_path, select_index, num_avg, number_from_select, num_subkey = 16, num_round = 10)
    generate_ct(ct_save, plaintext, key, num_subkey = 16)
    
      

In [9]:
if __name__ == "__main__":
    
    total_nunmber = 11000
    select_number = 10000
    initial_number = 0
    save_folder = 'Data/12m_9174_11k-1/'
    load_folder = 'Raw_data/12m_9174_11k-1/'     
    number_from_select = 1
    #每次增加的trace平均次数
    step_1 = 99
    
    for num_avg in range(1, 101, step_1):
        print("Start generating traces with repetiton: ", num_avg)
        trace_process(total_nunmber, select_number, initial_number, save_folder, load_folder, num_avg, number_from_select)
        

Start generating traces with repetiton:  1


100%|█████████████████| 10000/10000 [02:45<00:00, 60.36it/s]


=========== Trace Generated =============
=========== Plaintexts Generated =============


100%|████████████| 10000/10000 [00:00<00:00, 9974563.61it/s]


[[  0  17  34 ... 221 238 255]
 [  0  17  34 ... 221 238 255]
 [  0  17  34 ... 221 238 255]
 ...
 [  0  17  34 ... 221 238 255]
 [  0  17  34 ... 221 238 255]
 [  0  17  34 ... 221 238 255]]


...

[[  0  17  34 ... 221 238 255]
 [  0  17  34 ... 221 238 255]
 [  0  17  34 ... 221 238 255]
 ...
 [  0  17  34 ... 221 238 255]
 [  0  17  34 ... 221 238 255]
 [  0  17  34 ... 221 238 255]]


100%|██████████████| 10000/10000 [00:00<00:00, 17833.42it/s]


[[ 54 208  36 ... 186 182 187]
 [ 54 208  36 ... 186 182 187]
 [ 54 208  36 ... 186 182 187]
 ...
 [ 54 208  36 ... 186 182 187]
 [ 54 208  36 ... 186 182 187]
 [ 54 208  36 ... 186 182 187]]
=========== Key Generated =============


100%|████████████████| 10000/10000 [00:11<00:00, 908.18it/s]

=========== Ciphertexts Generated =============
